# Monad programming examples

Anton Antonov   
MathematicaForPrediction at WordPress  
RakuForPrediction at WordPress   
November 2025

----

## Introduction

### Context

Before going further, let us list the applications of monadic programming we consider:

1. Graceful failure handling

2. Rapid specification of computational workflows

3. Algebraic structure of written code

**Remark:** Those applications are discussed in [AAv5] (and its future Raku version.)

As a tools maker for Data Science (DS) and Machine Learning (ML), I am very interested in Point 1; but as a "simple data scientist" I am mostly interested in Point 2.

That said, a large part of my Raku programming has been dedicated to rapid and reliable code generation for DS and ML by leveraging the algebraic structure of corresponding software monads, i.e. Point 3. (See [AAv2, AAv3, AAv4].) For me, first and foremost, ***monadic programming pipelines are just convenient interfaces to computational workflows***. Often I make software packages that allow "easy", linear workflows that can have very involved computational steps and multiple tuning options.

### Dictionary

- **Monadic programming**   
  A method for organizing computations as a series of steps, where each step generates a value along with additional information about the computation, such as possible failures, non-determinism, or side effects. See [Wk1].

- **Monadic pipeline**   
  Chaining of operations with a certain syntax. Monad laws apply loosely (or strongly) to that chaining. 

- **Uniform Function Call Syntax (UFCS)**  
  A feature that allows both free functions and member functions to be called using the same `object.function()` method call syntax. 

- **Method-like call**   
  Same as UFCS. A Raku example: `[3, 4, 5].&f1.$f2`.

---- 

## Data wrangling

One appealing way to show that monadic pipelines result in clean and readable code, is to demonstrate their use in Raku through data wrangling operations.
Here we load "data packages", get the Titanic dataset, show its structure, and show a sample of its rows:

In [ ]:
#% html
use Data::Reshapers;
use Data::TypeSystem;
use Data::Translators;

my @dsTitanic = get-titanic-dataset();
say deduce-type(@dsTitanic);
@dsTitanic.pick(6) 
==> to-html(field-names => <id passengerClass passengerSex passengerAge passengerSurvival>)

Here is an `andthen` data wrangling monadic pipeline, the lines of which have the following interpretations:

- Initial pipeline value (the dataset)
- Rename columns
- Filter rows (with age greater or equal to 10)
- Group by the values of the columns "sex" and "survival"
- Show the structure of the pipeline value
- Give the sizes of each group as a result

In [ ]:
@dsTitanic 
andthen rename-columns($_,  {passengerAge => 'age', passengerSex => 'sex', passengerSurvival => 'survival'})
andthen $_.grep(*<age> ≥ 10).List
andthen group-by($_, <sex survival>)
andthen {say deduce-type($_); $_}($_)
andthen $_».elems

**Remark:** The `andthen` pipeline corresponds to the R pipeline in the second section.

Similar result can be obtained via cross-tabulation and using a pipeline with the feed (`==>`) operator:

In [ ]:
@dsTitanic
==> { .grep(*<passengerAge> ≥ 10) }()
==> { cross-tabulate($_, 'passengerSex', 'passengerSurvival') }()
==> to-pretty-table()

----

## Data wrangling code with multiple languages and packages

----

## Recommendation pipeline

----

## Functional parsers (multi-operation pipelines)

In can be said that the package ["FunctionalParsers"](https://raku.land/zef:antononcube/FunctionalParsers), [AAp4], implements multi-operator monadic pipelines the creation of parsers and interpreters. "FunctionalParsers" achieves that using special infix implementations.

In [48]:
use FunctionalParsers :ALL;
my &p1 = {1} ⨀ symbol('one');
my &p2 = {2} ⨀ symbol('two');
my &p3 = {3} ⨀ symbol('three');
my &p4 = {4} ⨀ symbol('four');
my &pH = {10**2} ⨀ symbol('hundred');
my &pT = {10**3} ⨀ symbol('thousand');
my &pM = {10**6} ⨀ symbol('million');
sink my &pNoun = symbol('things') ⨁ symbol('objects');

Here is a parser -- all three monad operations are used:

In [49]:
# Parse sentences that have (1) a digit part, (2) a multiplier part, and (3) a noun
my &p = (&p1 ⨁ &p2 ⨁ &p3 ⨁ &p4) ⨂ (&pT ⨁ &pH ⨁ &pM) ⨂ &pNoun;

# Interpreter:
# (1) flatten the parsed elements
# (2) multiply the first two elements and make a sentence with third element
sink &p = { "{$_[0] * $_[1]} $_[2]"} ⨀ {.flat} ⨀ &p 

Here the parser is applied to different sentences:

In [50]:
['three million things', 'one hundred objects', 'five thousand things']
andthen .map({ &p($_.words.List).head.tail })
andthen (.say for |$_)

3000000 things
100 objects
Nil


The last sentence is not parsed because `&p` knows only the digits from 1 to 4.

----

## References

### Articles, blog posts

[Wk1] Wikipedia entry: [Monad (functional programming)](https://en.wikipedia.org/wiki/Monad_(functional_programming)), URL: [https://en.wikipedia.org/wiki/Monad_(functional_programming)](https://en.wikipedia.org/wiki/Monad_(functional_programming)) . 

[Wk2] Wikipedia entry: [Monad transformer](https://en.wikipedia.org/wiki/Monad_transformer), URL: [https://en.wikipedia.org/wiki/Monad_transformer](https://en.wikipedia.org/wiki/Monad_transformer) .

[H1] Haskell.org article: [Monad laws,](https://wiki.haskell.org/Monad_laws) URL: [https://wiki.haskell.org/Monad_laws](https://wiki.haskell.org/Monad_laws). 

[SH2] Sheng Liang, Paul Hudak, Mark Jones, ["Monad transformers and modular interpreters",](http://haskell.cs.yale.edu/wp-content/uploads/2011/02/POPL96-Modular-interpreters.pdf) (1995), Proceedings of the 22nd ACM SIGPLAN-SIGACT symposium on Principles of programming languages. New York, NY: ACM. pp. 333--343. doi:10.1145/199448.199528.

[PW1] Philip Wadler, ["The essence of functional programming"](https://page.mi.fu-berlin.de/scravy/realworldhaskell/materialien/the-essence-of-functional-programming.pdf), (1992), 19'th Annual Symposium on Principles of Programming Languages, Albuquerque, New Mexico, January 1992.

[RW1] Hadley Wickham et al., [dplyr: A Grammar of Data Manipulation](https://github.com/tidyverse/dplyr), (2014), [tidyverse at GitHub](https://github.com/tidyverse), URL: [https://github.com/tidyverse/dplyr](https://github.com/tidyverse/dplyr) .
       (See also, [http://dplyr.tidyverse.org](http://dplyr.tidyverse.org) .)

[AA1] Anton Antonov, ["Monad code generation and extension"](https://mathematicaforprediction.wordpress.com/2017/06/23/monad-code-generation-and-extension/), (2017), [MathematicaForPrediction at WordPress](https://mathematicaforprediction.wordpress.com).

### Packages

[AAp1] Anton Antonov, [MonadMakers](https://resources.wolframcloud.com/PacletRepository/resources/AntonAntonov/MonadMakers/), Wolfram Language paclet, (2023), [Wolfram Language Paclet Repository](https://resources.wolframcloud.com/PacletRepository/).

[AAp2] Anton Antonov, [StatStateMonadCodeGeneratoreNon](https://github.com/antononcube/R-packages/tree/master/StateMonadCodeGenerator), R package, (2019-2024), 
[GitHub/@antononcube](https://github.com/antononcube/).

[AAp3] Anton Antonov, [DSL::English::DataQueryWorkflows](https://github.com/antononcube/Raku-DSL-English-DataQueryWorkflows), Raku package, (2020-2024), 
[GitHub/@antononcube](https://github.com/antononcube/).

[AAp4] Anton Antonov, [FunctionalParsers](https://github.com/antononcube/Raku-FunctionalParsers), Raku package, (2023-2024), 
[GitHub/@antononcube](https://github.com/antononcube/).

### Videos

[AAv1] Anton Antonov, [Monadic Programming: With Application to Data Analysis, Machine Learning and Language Processing](https://www.youtube.com/watch?v=_cIFA5GHF58), (2017), Wolfram Technology Conference 2017 presentation. [YouTube/WolframResearch](https://www.youtube.com/@WolframResearch).

[AAv2] Anton Antonov, [Raku for Prediction](https://www.youtube.com/watch?v=frpCBjbQtnA), (2021), [The Raku Conference 2021](https://www.youtube.com/@therakuconference6823).

[AAv3] Anton Antonov, [Simplified Machine Learning Workflows Overview](https://www.youtube.com/watch?v=Xy7eV8wRLbE), (2022), Wolfram Technology Conference 2022 presentation. [YouTube/WolframResearch](https://www.youtube.com/@WolframResearch).

[AAv4] Anton Antonov, [Simplified Machine Learning Workflows Overview (Raku-centric)](https://www.youtube.com/watch?v=p3iwPsc6e74), (2022), Wolfram Technology Conference 2022 presentation. [YouTube/@AAA4prediction](https://www.youtube.com/@AAA4prediction).

[AAv5] Anton Antonov, [Applications of Monadic Programming, Part 1, Questions & Answers](https://www.youtube.com/watch?v=Xz5B4B0kVco), (2025), [YouTube/@AAA4prediction](https://www.youtube.com/@AAA4prediction).
